In [13]:
import pandas as pd
import os
import statistics
from matplotlib import pyplot as plt
import seaborn as sns
from Readersourcing import Readersourcing
import numpy as np
from scipy.stats.stats import pearsonr
from scipy.stats.stats import kendalltau

plt.ioff()

In [16]:
# ------------------------------
# ---------- EXP 1-A -----------
# ------------------------------

# Analizzo l'andamento degli score di tre reader "speciali" per X shuffle dei rating di un dato seed.
# Reader 1 vota sempre nel modo "esatto" (sulla media). 
# Reader 2 vota sempre a metà scala rispetto a 1001.
# Reader 3 vota sempre sugli estremi.
# L'ordinamento desiderato degli score è Reader 1 -> Reader 2 -> Reader 3 (ordine crescente)

seed = Readersourcing(
    dataset_name="seed_shuffle_1_special", 
    dataset_folder_path="../data/{}/", 
    data_shuffled=True, 
    current_shuffle = 0,
    shuffle_amount=100
)

# Plot parameters

plt.rcParams['figure.figsize']=(6,6)
linesize = .5

# Paths

result_folder_experiments_path = "{}/experiments/".format(seed.result_folder_base_path)
result_folder_exp_1_a_path = "{}/experiment_1-a/".format(result_folder_experiments_path)
os.makedirs(result_folder_experiments_path, exist_ok=True)
os.makedirs(result_folder_exp_1_a_path, exist_ok=True)

In [17]:
shuffle_perc = 5
papers_perc = 5
df = seed.build_quantity_df_shuffle("Reader Score", shuffle_perc, 0)

readers_number = df[df["Shuffle"] == 0].shape[0]

df = df[
    (df["Identifier"] == readers_number-1) |
    (df["Identifier"] == readers_number-2) |
    (df["Identifier"] == readers_number-3) 
]

0/100 (0/100%)


ValueError: Expected object or value

In [73]:
with plt.rc_context({'lines.linewidth': linesize}):
    ax = sns.pointplot(x="Identifier", y="Quantity", hue="Shuffle", 
                       data=df, order=[readers_number-3,readers_number-2,readers_number-1])
    ax.set_title(f"shuffle_perc: {shuffle_perc}%")
fig = ax.get_figure()
path = "{}/special-reader-score-lineplot.pdf".format(result_folder_exp_1_a_path)
fig.savefig(path, bbox_inches='tight')
plt.close(fig)

In [55]:
# ------------------------------
# ---------- EXP 1-B -----------
# ------------------------------

# Analizzo le quantità calcolate da Readersourcing per X shuffle dei rating di un dato seed.
# Per ciascuna quantità viene costruito un grafico che mostra i valori della quantità corrente per ciascuno shuffle.

seed_shuffle_1 = Readersourcing(
    dataset_name="seed_shuffle_1", 
    dataset_folder_path="../data/{}/", 
    data_shuffled=True, 
    current_shuffle = 0,
    shuffle_amount=100
)

# Plot parameters

linesize = .75

# Paths

result_folder_experiments_path = "{}/experiments/".format(seed_shuffle_1.result_folder_base_path)
result_folder_exp_1_b_path = "{}/experiment_1-b/".format(result_folder_experiments_path)
os.makedirs(result_folder_experiments_path, exist_ok=True)
os.makedirs(result_folder_exp_1_b_path, exist_ok=True)

In [56]:
shuffle_perc = 5
papers_perc = 5
df = seed.build_quantity_df_shuffle("Paper Score", seed_shuffle_1, shuffle_perc, papers_perc)

print("---------- BUILDING PAPER SCORE LINE PLOT ----------")
with plt.rc_context({'lines.linewidth': linesize}):
    ax = sns.pointplot(x="Identifier", y="Quantity", hue="Shuffle", data=df)
    ax.set_title(f"shuffle_perc: {shuffle_perc}%, papers_perc: {papers_perc}%")
fig = ax.get_figure()
path = "{}/paper-score-lineplot.pdf".format(result_folder_exp_1_b_path)
fig.savefig(path, bbox_inches='tight')
plt.close(fig)

0/100 (0/100%)
100/100 (100/100%)
---------- BUILDING PAPER SCORE LINE PLOT ----------


In [57]:
shuffle_perc = 5
papers_perc = 5
df = seed.build_quantity_df_shuffle("Paper Steadiness", seed_shuffle_1, shuffle_perc, papers_perc)

print("---------- BUILDING PAPER STEADINESS LINE PLOT ----------")
with plt.rc_context({'lines.linewidth': linesize}):
    ax = sns.pointplot(x="Identifier", y="Quantity", hue="Shuffle", data=df)
    ax.set_title(f"shuffle_perc: {shuffle_perc}%, papers_perc: {papers_perc}%")
fig = ax.get_figure()
path = "{}/paper-steadiness-lineplot.pdf".format(result_folder_exp_1_b_path)
fig.savefig(path, bbox_inches='tight')
plt.close(fig)

0/100 (0/100%)
100/100 (100/100%)
---------- BUILDING PAPER STEADINESS LINE PLOT ----------


In [58]:
shuffle_perc = 5
readers_perc = 2
df = seed.build_quantity_df_shuffle("Reader Score", seed_shuffle_1, shuffle_perc, readers_perc)

print("---------- BUILDING READER SCORE LINE PLOT ----------")
with plt.rc_context({'lines.linewidth': linesize}):
    ax = sns.pointplot(x="Identifier", y="Quantity", hue="Shuffle", data=df)
    ax.set_title(f"shuffle_perc: {shuffle_perc}%, readers_perc: {readers_perc}%")
fig = ax.get_figure()
path = "{}/reader-score-lineplot.pdf".format(result_folder_exp_1_b_path)
fig.savefig(path, bbox_inches='tight')
plt.close(fig)

0/100 (0/100%)
100/100 (100/100%)
---------- BUILDING READER SCORE LINE PLOT ----------


In [60]:
shuffle_perc = 5
readers_perc = 2
df = seed.build_quantity_df_shuffle("Reader Steadiness", seed_shuffle_1, shuffle_perc, readers_perc)

print("---------- BUILDING READER STEADINESS LINE PLOT ----------")
with plt.rc_context({'lines.linewidth': linesize}):
    ax = sns.pointplot(x="Identifier", y="Quantity", hue="Shuffle", data=df)
    ax.set_title(f"shuffle_perc: {shuffle_perc}%, readers_perc: {readers_perc}%")
fig = ax.get_figure()
path = "{}/reader-steadiness-lineplot.pdf".format(result_folder_exp_1_b_path)
fig.savefig(path, bbox_inches='tight')
plt.close(fig)

0/100 (0/100%)
100/100 (100/100%)
---------- BUILDING READER STEADINESS LINE PLOT ----------


In [34]:
# ------------------------------
# ---------- EXP 1-C -----------
# ------------------------------

# Le quantità calcolate da Readersourcing vengono comparate con la loro media aritmetica, geometrica, bayesiana e con la mediana.
# Gli indici standard vengono calcolati sui giudizi ricevuti da ciascun Paper e sui giudizi assegnati da ciascun Reader.

seed = Readersourcing(
    dataset_name="ground_truth_2", 
    dataset_folder_path="../data/{}/", 
)

# Plot parameters

linesize = .75
    
# Paths

result_folder_experiments_path = "{}/experiments/".format(seed.result_folder_base_path)
result_folder_exp_1_c_path = "{}/experiment_1-c/".format(result_folder_experiments_path)
os.makedirs(result_folder_experiments_path, exist_ok=True)
os.makedirs(result_folder_exp_1_c_path, exist_ok=True)

# Functions

def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))

def compute_means(raw_df, quantity_df, quantity_label, axis):
    
    mean_df = pd.DataFrame(columns=[quantity_label])
    mean_df[quantity_label] = quantity_df["Quantity"]
    mean_df["Arithmetic Mean"] = raw_df.mean(axis=axis)
    mean_df["Geometric Mean"] = raw_df.apply(geo_mean, axis=axis)
    mean_df["Median"] = raw_df.mean(axis=axis)
    cols = []
    for col in raw_df.columns.values:
        cols.append(col)
    raw_df["v"] = raw_df[cols].count(axis=1)    
    m = np.mean(raw_df['v'])
    raw_df['w'] = raw_df['v']/(raw_df['v'] + m)
    raw_df['r'] = np.mean(raw_df[cols], axis=1)
    c = np.nanmean(raw_df[cols].values.flatten())
    raw_df['b'] = raw_df['w']*raw_df['r'] + (1 - raw_df['w'])*c
    raw_df = raw_df.drop(['v', 'w', 'r'], axis=1)
    mean_df["Bayesian Mean"] = raw_df["b"]
    
    corr_df = pd.DataFrame(columns=[
        "Rho (Arithmetic Mean)",
        "Rho (Geometric Mean)",
        "Rho (Bayesian Mean)",
        "Rho (Median)",
        "Tau (Arithmetic Mean)",
        "Tau (Geometric Mean)",
        "Tau (Bayesian Mean)",
        "Tau (Median)",
    ])
    
    dict = {}
    quantity_column = mean_df[quantity_label]
    columns = mean_df.drop(quantity_label, axis=1).columns
    for col in columns:
        mean_column = mean_df[col]
        pearson_corr_val = pearsonr(quantity_column, mean_column)[0]
        kendall_corr_val = kendalltau(quantity_column, mean_column)[0]
        dict[f'Rho ({col})'] = pearson_corr_val
        dict[f'Tau ({col})'] = kendall_corr_val
    corr_df = corr_df.append(dict, ignore_index=True)
    
    return corr_df
    
ratings_df = seed.extract_ratings_df()
paper_score_df = seed.build_quantity_df("Paper Score", 100)
reader_score_df = seed.build_quantity_df("Reader Score", 100)
corr_df_1 = compute_means(ratings_df, paper_score_df, "Paper Score", 0)
corr_df_2 = compute_means(ratings_df, reader_score_df, "Reader Score", 1)

display(corr_df_1)
display(corr_df_2)

0/0 (100/100%)
0/0 (100/100%)


,Rho (Arithmetic Mean),Rho (Geometric Mean),Rho (Bayesian Mean),Rho (Median),Tau (Arithmetic Mean),Tau (Geometric Mean),Tau (Bayesian Mean),Tau (Median)
0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0


,Rho (Arithmetic Mean),Rho (Geometric Mean),Rho (Bayesian Mean),Rho (Median),Tau (Arithmetic Mean),Tau (Geometric Mean),Tau (Bayesian Mean),Tau (Median)
0,-0.241703,0.012493,-0.241703,-0.241703,-0.333333,-0.333333,-0.333333,-0.333333
